In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,Activation
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
# load dataset
#binary_df = pd.read_csv('/content/MyDrive/MyDrive/csv/binary.csv')
img_path = ('/content/MyDrive/MyDrive/dataset/chest xray256')

In [ ]:
#get labels from csv_file
class_names = sorted(binary_df.label.unique())

class_names

#put classes in dict 
classes = dict(zip(list(range(2)),class_names))
classes

#get name by code
def get_class_name(key):
    for ke , nam in classes.items():
        if key == ke :
            return nam

In [ ]:
classes

{0: 'Finding', 1: 'No finding'}

In [ ]:
#get label of image
def get_class_name(key):
    for ke , nam in classes.items():
        if key == ke :
            return nam

In [ ]:
#file_names = os.listdir(data_path)
#label = []
#for filename in file_names:
#    labels = filename.split('.')[0]
#    labels = get_class_name(labels)
#    if labels == 'No finding':
#        label.append('Normal')
#    else:
#        label.append('Not')
#binary_df = pd.DataFrame({'image_id':file_names,'label':label})        
        

In [ ]:
#binary_df.to_csv('/content/MyDrive/MyDrive/binary.csv')

In [ ]:
#binary_df.value_counts('label')

In [ ]:
train_df = pd.DataFrame({'image_id':x_train,'label':y_train})
test_df = pd.DataFrame({'image_id':x_test,'label':y_test})

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(binary_df,
        img_path,
        x_col = 'image_id',
        y_col = 'label',  
        target_size=(256, 256), 
        batch_size=10,
        class_mode='binary')
validation_generator = test_datagen.flow_from_dataframe(binary_df,
        img_path,
        x_col = 'image_id',
        y_col = 'label',
        target_size=(256, 256),
        batch_size=10,
        class_mode='binary')

Found 8800 validated image filenames belonging to 2 classes.
Found 8800 validated image filenames belonging to 2 classes.


In [ ]:
def generateFolds(n_splits = None):
    kf = KFold(n_splits= n_splits)
    for id,(tr_,val_) in enumerate(kf.split(binary_df["image_id"],binary_df["label"])):
        binary_df.loc[val_,'kfold'] = int(id)
    return binary_df["kfold"].astype(int)
generateFolds(10)
#get model name 
def get_model_name(k):
    return 'model_'+str(k)+'.hdf5'

In [ ]:
for fold in range(10):
  print(fold)
  X_train = binary_df[binary_df.kfold!=fold].drop('kfold',axis=1)
  X_val = binary_df[binary_df.kfold==fold].drop('kfold',axis=1)
  print(len(X_train),len(X_val))

train = train_datagen.flow_from_dataframe(X_train,
                                            img_path,
                                            x_col='image_id',
                                            y_col='label',
                                            target_size=(512,512),
                                            class_mode='binary',
                                            batch_size=33)
X_train

In [ ]:
# prepare csv file for cross_valdition
csv='/content/Mydrive/MyDrive/csv/binary.csv'
dataf = pd.read_csv(csv)
find = dataf.iloc[0:4400,1].values
no_find = dataf.iloc[4400:,1].values
lab_find = dataf.iloc[0:4400,2].values
lab_no_find = dataf.iloc[4400:,2].values
id = []
name = []
count = 0 
count_2 = 0
for i in range(0,8800):
  if i % 2 == 0:
    id.append(find[count_2])
    name.append(lab_find[count_2])
    count_2 += 1
  else :
    id.append(no_find[count])
    name.append(lab_no_find[count])
    count +=1  


class_nam = []
for i in name:
  if i == 'No finding':
    class_nam.append(i)
  else :
    class_nam.append('Finding')
class_nam      
df = pd.DataFrame({'image_id':id,'label':class_nam})   
df.to_csv('/content/Mydrive/MyDrive/csv/cross_val.csv')         

2

In [ ]:
binary_df = pd.read_csv('/content/MyDrive/MyDrive/csv/cross_val.csv')

In [ ]:
x = binary_df['image_id']
y = binary_df['label']

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

acc_per_fold = []
loss_per_fold = []
for train,test in kfold.split(x, y):

  model = Sequential()
  model.add(Conv2D(128,(3,3),input_shape=(256,256,1),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(32,(3,3),activation='relu'))
  model.add(Conv2D(32,(3,3),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Conv2D(32,(3,3),activation='relu'))
  model.add(Conv2D(32,(3,3),activation='relu')) 
  model.add(MaxPooling2D(pool_size=(4, 4)))
  
  model.add(Flatten()) 
  model.add(Dense(128,activation='relu'))
  model.add(Dense(1 , activation = 'sigmoid'))
  
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  
  
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  


  train_gen = train_datagen.flow_from_directory(img_path,
                                            target_size=(256,256),
                                            class_mode='binary',
                                            batch_size=32)
  val_gen = test_datagen.flow_from_dataframe(img_path,
                                         target_size=(256, 256),
                                         batch_size=32,
                                         class_mode='binary')
  # Fit data to model
  model.fit(train_gen[train],epochs = 10,validation_data=val_gen[test])

  # Generate generalization metrics
  model.save('/content/drive/MyDrive/save_model512/' + get_model_name(fold)+'.h5')

  # Increase fold number
  fold_no = fold_no + 1


------------------------------------------------------------------------
Training for fold 1 ...
Found 7920 validated image filenames belonging to 2 classes.
Found 880 validated image filenames belonging to 2 classes.
Epoch 1/10


NotFoundError: ignored

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator) //10,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=len(validation_generator) // 10,verbose =1)

In [ ]:
  model.save('/content/MyDrive/MyDrive/first_try_binary.h5')

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/first_try_binary.h5')

In [ ]:
from keras.preprocessing.image import img_to_array

In [ ]:
test_image = load_img('/content/MyDrive/MyDrive/chest xray256/000434271f63a053c4128a0ba6352c7f.jpg', target_size = (256, 256))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
pred = model.predict(test_image)
get_class_name(pred)

'Normal'

0